In [428]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import os, sys, fnmatch
import cv2, time, math, pickle
import numpy as np
import argparse, pprint
import pyrealsense2 as rs

from sklearn import linear_model, datasets
from matplotlib import pyplot as plt

from scripts.hyutils.sys_utils import *
from scripts.hyutils.img_utils import *
from scripts.hyutils.debug_utils import *
from scripts.hyutils.plotting_utils import *
from scripts.hyutils.uv_mapping_utils import *
from scripts.devices.d415_camera import *
from scripts.VBOATS import VBOATS

import ipywidgets as widgets
from IPython.display import display, HTML, Math
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual,interact, HBox, Layout,VBox

pp = pprint.PrettyPrinter(indent=4)
homeDir = os.path.abspath('')

vboat = VBOATS()
vboat.debug = False
vboat.dead_x = 0
vboat.dead_y = 3


%load_ext cython

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
cam.__del__()

In [ ]:
cam = CameraD415(flag_save=False, use_statistics=False, fps=30,verbose=True)

intr = cam.get_intrinsics()
extr = cam.get_extrinsics()
focal = intr["depth"].fx
baseline = extr.translation[0]
# print(focal, baseline)

In [ ]:
rgb,depth = cam.read()
pplot(cv2.cvtColor(rgb,cv2.COLOR_RGB2BGR),"hey")
pplot(depth,"depth")

In [777]:
prevM = None;    prevB = None
h = None;        w = None

def getNewImages():
    global cam,focal,baseline
    rgb,_depth = cam.read()
    tmp = _depth/65535.0
    depth = np.uint8(tmp*255)
    depth = cv2.cvtColor(depth,cv2.COLOR_GRAY2BGR)

    tmp2 = _depth*0.001
    loc = np.where(tmp2 == 0.0)
    tmp2[loc] = 1.0
    disparity = (focal*baseline)/tmp2
    disparity[loc] = 0.0
    disparity2uintGain = 255/np.max(disparity)
    disparity = np.uint8(disparity*disparity2uintGain)
    return np.copy(rgb), np.copy(depth), np.copy(disparity)

def estimateGndLineCoeffs(filtered_vmap, stopP = 0.85, maxTrials=30,verbose = False,show_plots=False):
    gndImg = np.copy(filtered_vmap)
    nonzero = gndImg.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    nonzerox = np.reshape(nonzerox,(nonzerox.shape[0],1))

    ransac = linear_model.RANSACRegressor(stop_probability=stopP,max_trials=maxTrials)
    ransac.fit(nonzerox, nonzeroy)
    inlier_mask = ransac.inlier_mask_
    outlier_mask = np.logical_not(inlier_mask)

    line_X = np.arange(nonzerox.min(), nonzerox.max())[:, np.newaxis]
    line_y_ransac = ransac.predict(line_X)

    if(show_plots):
        plt.figure("RANSAC")
        plt.clf()
        plt.subplots_adjust(wspace=0.0,hspace=0.0,left=0.0,right=1.0,top=1.0, bottom=0.0)
        plt.axis([0.0, 255.0, 480, 0])
        plt.imshow(gndImg,interpolation='bilinear')
        plt.scatter(nonzerox,nonzeroy, color='yellowgreen', marker='.',label='Inliers')
        plt.plot(line_X, line_y_ransac, color='red', linewidth=2, label='RANSAC regressor')
        plt.show()

    m = ransac.estimator_.coef_
    b = ransac.estimator_.intercept_
    if(verbose): print("Estimated coefficients (RANSAC): m = %.3f | b = %.3f" % (m,b))
    return m,b, line_X,line_y_ransac

def filterVmap(_vmap, thresholds = [0.85,0.85,0.75,0.5], verbose=False, debug=False):
    raw_vmap = np.copy(_vmap)
    vMax = np.max(_vmap)
    nThreshs = int(math.ceil((vMax/256.0)) * len(thresholds))
    if(debug): print("vMax, nThreshs" % (vMax, nThreshs))

    # Rough Theshold top of vmap heavy prior
    h, w = raw_vmap.shape[:2]
    dh = h / 3
    topV = raw_vmap[0:dh, :]
    botV = raw_vmap[dh:h, :]
    tmpThresh = int(np.max(topV)*0.05)
    _,topV = cv2.threshold(topV, tmpThresh,255,cv2.THRESH_TOZERO)
    preVmap = np.concatenate((topV,botV), axis=0)

    stripsPV = []
    stripsV = strip_image(preVmap, nstrips=nThreshs, horizontal_strips=False)
    for i, strip in enumerate(stripsV):
        tmpMax = np.max(strip)
        tmpMean = np.mean(strip)
        tmpStd = np.std(strip)
        if(tmpMean == 0):
            stripsPV.append(strip)
            continue
        if(verbose): print("---------- [Strip %d] ---------- \r\n\tMax = %.1f, Mean = %.1f, Std = %.1f" % (i,tmpMax,tmpMean,tmpStd))
        dratio = vMax/255.0
        relRatio = (tmpMax-tmpStd)/float(vMax)
        rrelRatio = (tmpMean)/float(tmpMax)
        if(verbose): print("\tRatios: %.3f, %.3f, %.3f" % (dratio, relRatio, rrelRatio))
        if(relRatio >= 0.4): gain = relRatio + rrelRatio
        else: gain = 1.0 - (relRatio + rrelRatio)
        thresh = int(thresholds[i]* gain * tmpMax)
        if(verbose): print("\tGain = %.2f, Thresh = %d" % (gain,thresh))
        _, tmpStrip = cv2.threshold(strip, thresh,255,cv2.THRESH_TOZERO)
        # _, tmpStrip = cv2.threshold(strip, thresh,255,cv2.THRESH_BINARY)
        stripsPV.append(tmpStrip)

    filtV = np.concatenate(stripsPV, axis=1)
    return filtV, stripsV, stripsPV

def filterUmap(_umap, thresholds = [0.85,0.85,0.75,0.5], verbose=False, debug=False):
    raw_umap = np.copy(_umap)
    uMax = np.max(_umap)
    nThreshs = int(math.ceil((uMax/256.0)) * len(thresholds))
    if(debug): print("uMax, nThreshs" % (uMax, nThreshs))

    # Rough Theshold top of vmap heavy prior
#     h, w = raw_raw_umapvmap.shape[:2]
#     dh = h / 3
#     topV = raw_umap[0:dh, :]
#     botV = raw_umap[dh:h, :]
#     tmpThresh = int(np.max(topV)*0.25)
#     _,topV = cv2.threshold(topV, tmpThresh,255,cv2.THRESH_TOZERO)
#     preUmap = np.concatenate((topV,botV), axis=0)
    _,preUmap = cv2.threshold(raw_umap, 2,255,cv2.THRESH_TOZERO)

    stripsPU = []
    stripsU = strip_image(preUmap, nstrips=nThreshs)
    for i, strip in enumerate(stripsU):
        tmpMax = np.max(strip)
        tmpMean = np.mean(strip)
        tmpStd = np.std(strip)
        if(tmpMean == 0):
            stripsPU.append(strip)
            continue
        if(verbose): print("---------- [Strip %d] ---------- \r\n\tMax = %.1f, Mean = %.1f, Std = %.1f" % (i,tmpMax,tmpMean,tmpStd))
        dratio = uMax/255.0
        relRatio = (tmpMax-tmpStd)/float(uMax)
        rrelRatio = (tmpMean)/float(tmpMax)
        if(verbose): print("\tRatios: %.3f, %.3f, %.3f" % (dratio, relRatio, rrelRatio))
        if(relRatio >= 0.4): gain = relRatio + rrelRatio
        else: gain = 1.0 - (relRatio + rrelRatio)
        thresh = int(thresholds[i]* gain * tmpMax)
        if(verbose): print("\tGain = %.2f, Thresh = %d" % (gain,thresh))
        _, tmpStrip = cv2.threshold(strip, thresh,255,cv2.THRESH_TOZERO)
        # _, tmpStrip = cv2.threshold(strip, thresh,255,cv2.THRESH_BINARY)
        stripsPU.append(tmpStrip)

    filtU = np.concatenate(stripsPU, axis=0)
    return filtU, stripsU, stripsPU


def smoothUnstableGndLine(curCoeffs, prevCoeffs, kds=(0.05,0.5), verbose=False, debug=False):
    print("curM, prevM: %.3f, %.3f" % (curCoeffs[0], prevCoeffs[0]))
    print("curB, prevB: %.3f, %.3f" % (curCoeffs[1], prevCoeffs[1]))
    errorM = prevCoeffs[0] - curCoeffs[0]
    errorB = prevCoeffs[1] - curCoeffs[1]
    print("errorM = %.3f | errorB = %.3f" % (errorM, errorB))
    dSlopes = math.fabs(curCoeffs[0] - prevCoeffs[0])
    dIntercepts = math.fabs(curCoeffs[1] - prevCoeffs[1])
    print("dSlope = %.3f | dIntercept = %.3f" % (dSlopes, dIntercepts))
    
    mDGain = dSlopes * kds[0]
    bDGain = dIntercepts * kds[1]
    print("mDGain = %.3f | bDGain = %.3f" % (mDGain,bDGain))
    newSlope = (prevCoeffs[0]) - mDGain
    newIntercept = prevCoeffs[1] - bDGain
#     newSlope = (errorM) - mDGain
#     newIntercept = errorB - bDGain
    print("newSlope = %.3f | newIntercept = %.3f" % (newSlope,newIntercept))

    return newSlope, newIntercept

def extractGndLine(slope, intercept, validSlopeThresh = 0.3, dslopeThresh = 2.0, dinterceptThresh = 300.0, interceptOffset = -15.0, verbose = False, debug = False):
    global prevM, prevB, raw_vmap
    
    if(prevM is None): prevM = slope
    if(prevB is None): prevB = intercept
    if(debug): print("Previous line coefficients = %.3f, %.3f" % (prevM,prevB))
    h,w = raw_vmap.shape[:2]
    validGnd = False

    # Only return a ground line if estimated slope is steep enough (i.e. try to prevent flat lines)
    if(slope >= validSlopeThresh):
        # Check for significant schanges in segmented ground line
        dSlope = math.fabs(prevM - slope)
        dB = math.fabs(prevB - intercept)
        if(debug): print("dSlope, dIntercept = %.3f, %.3f" % (dSlope,dB))
        # Filter out sudden jumps in segmented ground line, if warrented
        if((dSlope >= dslopeThresh) or (dB >= dinterceptThresh)):
            newSlope, newIntercept = smoothUnstableGndLine((slope,intercept),(prevM,prevB))
        else: newSlope = slope; newIntercept = intercept
        # Store coefficients for next iteration
        prevM = slope;               prevB = intercept
        # Extract ground line parameters
        yf = int(newSlope*w + newIntercept);  pt1 = (0,int(newIntercept));  pt2 = (w,int(yf+interceptOffset))
        # Create point array for removing anything below ground line (for removing noise)
        pts = np.array([(0, h),pt1,pt2,(w, h),(0, h)])
        validGnd = True
    else:
        prevM = None; prevB = None
        pts = None
        if(verbose): print("No Ground Detected.")
    return validGnd, pts

def removeGround(_vmap, _pts, debug_line_vis=False):
    img = np.copy(_vmap)
    color = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
    cv2.fillPoly(color, [pts], (0,0,0))
    if(debug_line_vis):
        ground_display = np.copy(_vmap)
        ground_display = cv2.applyColorMap(ground_display,cv2.COLORMAP_PARULA)
        cv2.line(ground_display,tuple(pts[1]), tuple(pts[2]), (0,255,255),1)
    else: ground_display = None
    return cv2.cvtColor(color,cv2.COLOR_BGR2GRAY), ground_display

In [461]:
%%cython -a
import cython
@cython.boundscheck(False)

cpdef unsigned char[:, :] hough_line_fast(unsigned char [:, :]accumulator,long [:] x_idxs,long [:]y_idxs, double [:] cos_t,double [:] sin_t, long diag_len,long num_thetas):
#     global diag_len,num_thetas,cos_t,sin_t
    cdef int i, x, y, rho
    for i in range(len(x_idxs)):
        x = x_idxs[i]
        y = y_idxs[i]

        for t_idx in range(num_thetas):
            # Calculate rho. diag_len is added for a positive index
            rho = diag_len + int(round(x * cos_t[t_idx] + y * sin_t[t_idx]))
            accumulator[rho, t_idx] += 1

    return accumulator

In [442]:
def hough_line(img, angle_step=1, lines_are_white=True, value_threshold=5):
    # Cache some resuable values
    cos_t = np.cos(thetas)
    sin_t = np.sin(thetas)
    num_thetas = len(thetas)

    # Hough accumulator array of theta vs rho
    accumulator = np.zeros((2 * diag_len, num_thetas), dtype=np.uint8)
    # (row, col) indexes to edges
    are_edges = img > value_threshold if lines_are_white else img < value_threshold
    y_idxs, x_idxs = np.nonzero(are_edges)

    # Vote in the hough accumulator
    for i in range(len(x_idxs)):
        x = x_idxs[i]
        y = y_idxs[i]

        for t_idx in range(num_thetas):
            # Calculate rho. diag_len is added for a positive index
            rho = diag_len + int(round(x * cos_t[t_idx] + y * sin_t[t_idx]))
            accumulator[rho, t_idx] += 1

    return accumulator, thetas, rhos

In [443]:
def show_hough_line(img, accumulator, thetas, rhos, save_path=None):
    fig, ax = plt.subplots(1, 2, figsize=(10, 10))

    ax[0].imshow(img, cmap=plt.cm.gray)
    ax[0].set_title('Input image')
    ax[0].axis('image')

    ax[1].imshow(accumulator, cmap='jet',extent=[np.rad2deg(thetas[-1]), np.rad2deg(thetas[0]), rhos[-1], rhos[0]])
    ax[1].set_aspect('equal', adjustable='box')
    ax[1].set_title('Hough transform')
    ax[1].set_xlabel('Angles (degrees)')
    ax[1].set_ylabel('Distance (pixels)')
    ax[1].axis('image')

    # plt.axis('off')
    if save_path is not None: plt.savefig(save_path, bbox_inches='tight')
    plt.show()

In [431]:
%%cython -a

def threshold_slow(T, image):
    # grab the image dimensions
    h = image.shape[0]
    w = image.shape[1]
    
    # loop over the image, pixel by pixel
    for y in range(0, h):
        for x in range(0, w):
            # threshold the pixel
            image[y, x] = 255 if image[y, x] >= T else 0
            
    # return the thresholded image
    return image

In [455]:
displayv = np.copy(vmapFiltered)
displayv = cv2.cvtColor(displayv, cv2.COLOR_GRAY2BGR)

t0 = time.time()
accumulator, thetas, rhos = hough_line(vmapFiltered)
t1 = time.time()
dt = t1 - t0
print("[INFO] vboat_testing_node::loop() --- Took %f seconds (%.2f Hz) to complete" % (dt, 1/dt))

show_hough_line(vmapFiltered, accumulator,thetas, rhos)

idx = np.argmax(accumulator)
rho = rhos[idx / accumulator.shape[1]]
theta = thetas[idx % accumulator.shape[1]]
print "rho={0:.2f}, theta={1:.0f}".format(rho, np.rad2deg(theta))
pt1, pt2 = extract_houghline(rho,theta)
cv2.line(displayv,pt1, pt2, (0,255,0),1)
pplot(displayv,"Ground Line")

[INFO] vboat_testing_node::loop() --- Took 2.337130 seconds (0.43 Hz) to complete
rho=19.52, theta=1


In [466]:
t0 = time.time()
img = np.copy(vmapFiltered)
lines_are_white=True
value_threshold=5
# Rho and Theta ranges
thetas = np.deg2rad(np.arange(-90.0, 90.0, 1))
width = 255
height = 480
diag_len = int(round(math.sqrt(width * width + height * height)))
rhos = np.linspace(-diag_len, diag_len, diag_len * 2)
# Cache some resuable values
cos_t = np.cos(thetas)
sin_t = np.sin(thetas)
num_thetas = len(thetas)
accumulator = np.zeros((2 * diag_len, num_thetas), dtype=np.uint8)
# (row, col) indexes to edges
are_edges = img > value_threshold if lines_are_white else img < value_threshold
y_idxs, x_idxs = np.nonzero(are_edges)

accumulator = hough_line_fast(accumulator,x_idxs,y_idxs, cos_t,sin_t, diag_len, num_thetas)

t1 = time.time()
dt = t1 - t0
print("[INFO] vboat_testing_node::loop() --- Took %f seconds (%.2f Hz) to complete" % (dt, 1/dt))

[INFO] vboat_testing_node::loop() --- Took 0.041757 seconds (23.95 Hz) to complete


In [77]:
tmpR = rhos[693]
tmpAng = (thetas[104])

pt1, pt2 = extract_houghline(tmpR,tmpAng)
cv2.line(display,pt1, pt2, (0,255,0),1)
pplot(display,"Ground Line")

Theta = 14.000 | Radius = 149.638


In [ ]:
# prevM = None;    prevB = None
rgb, depth, disparity = getNewImages()
raw_umap, raw_vmap, dt = vboat.get_uv_map(disparity)

vmapFiltered,stripsV,stripsPV = prefilterVmap(raw_vmap,[0.85,0.85,0.35,0.5])
m,b, xs,ys = groundSeg(vmapFiltered, show_plots=True)

validGnd, pts = extractGndLine(m,b)

if(validGnd):
    vmapNoGnd, vizGndLine = removeGround(raw_vmap,pts,True)    
    pplot(cv2.cvtColor(vizGndLine,cv2.COLOR_RGB2BGR),"Ground Line")
else:
    print("Ground Line too erratic")
#     vmapNoGnd = vizGndLine = raw_vmap

pplot(vmapFiltered,"Filtered Vmap")
pplots(stripsV,"Vmap Strips",(1,len(stripsV)))
pplots(stripsPV,"Filtered Vmap Strips",(1,len(stripsPV)))  

overlay = make_uv_overlay(rgb,raw_umap,raw_vmap)
pplot(cv2.cvtColor(overlay,cv2.COLOR_RGB2BGR),"overlay")

In [ ]:
%%cython -a
cimport numpy as np

def isGndPresentFast(found_lines,minDeg=-89.0, maxDeg=-26.0):
    cnt = 0
    summer = 0
    flag = False
    avgTheta = 0.0
    minAng = np.deg2radf(minDeg)
    maxAng = np.deg2radf(maxDeg)
    angs = found_lines[:,0,1]
    for theta in angs:
        angle = theta-np.NPY_PI
        if(minAng <= angle <= maxAng):
            summer += angle
            cnt += 1
    if(cnt > 0):
        avgTheta = summer/float(cnt)
        flag = True
    return flag, avgTheta

In [645]:
def isGndPresent(found_lines,minDeg=-89.0, maxDeg=-26.0):
    cnt = 0
    summer = 0
    flag = False
    avgTheta = 0.0
    minAng = np.deg2rad(minDeg)
    maxAng = np.deg2rad(maxDeg)
    angs = found_lines[:,0,1]
    for theta in angs:
        angle = theta-np.pi
        if(minAng <= angle <= maxAng):
            summer += angle
            cnt += 1
    if(cnt > 0):
        avgTheta = summer/float(cnt)
        flag = True
    return flag, avgTheta

def estimateGndLineCoeffsHough2(filtered_vmap,hough_thresh=100, deg_offset=2.0):
    sets = []
    validGnd = False
    bestM = 0.0
    bestIntercept = 0
    # print(filtered_vmap.shape)
    lines = cv2.HoughLines(filtered_vmap,1,np.pi/180, hough_thresh)

    # if(lines is None): lines = cv2.HoughLines(filtered_vmap,1,np.pi/180, 50)
    if(lines is None):
        print("couldnt find hough lines")
        return validGnd,bestM,bestIntercept, sets

    validGnd, avgTheta = isGndPresent(lines)
    if(validGnd):
        avgAngle = np.rad2deg(avgTheta)
        minAng = np.deg2rad(avgAngle - deg_offset)
        maxAng = np.deg2rad(avgAngle + deg_offset)
        h,w = filtered_vmap.shape[:2]
        for i in range(0,len(lines)):
            for r,theta in lines[i]:
                angle = theta-np.pi
                if(minAng <= angle <= maxAng):
                    a = np.cos(theta);            b = np.sin(theta)
                    x0 = a*r;                     y0 = b*r;
                    x1 = int(x0 + 1000*(-b));     y1 = int(y0 + 1000*(a))
                    x2 = int(x0 - 1000*(-b));     y2 = int(y0 - 1000*(a))
                    m = float(y2 - y1)/float(x2 - x1)
                    b = int(y1 - m * x1)
                    if(b > bestIntercept):
                        bestIntercept = b
                        bestM = m
                    # pt1 = (x1,y1)
                    # pt2 = (x2,y2)
                    # pts = [(0,h),pt1,pt2,(w, h),(0, h)]
                    # pts = np.array(pts)
                    # sets.append(pts)
    else: print("No Ground Found")
    return validGnd,bestM,bestIntercept, sets

In [531]:
# found_imgs, _ = grab_dir_images("/home/hunter/devel/vision_playground/images/depth")
found_imgs, _ = grab_dir_images("/home/hunter/devel/vision_playground/gnd_tests/depth")


sFImgs = sorted(found_imgs)
imgPathSelector = widgets.Select(options=sFImgs,value=str(sFImgs[0]),description='File:', layout=Layout(width='100%'))
curImg = raw_umap = raw_vmap = []
def thresholding(_img):
    global curImg,raw_umap,raw_vmap
    img = cv2.imread(_img,cv2.IMREAD_GRAYSCALE)
    curImg = np.copy(img)
    raw_umap, raw_vmap, _ = vboat.get_uv_map(curImg)
    cv2.rectangle(raw_vmap,(0,0),(3, raw_vmap.shape[0]),(0,0,0), cv2.FILLED)
    cv2.rectangle(raw_umap,(0,0),(raw_umap.shape[1], 3),(0,0,0), cv2.FILLED)
    overlay = make_uv_overlay(img,raw_umap,raw_vmap)
    pplot(cv2.cvtColor(overlay,cv2.COLOR_RGB2BGR),"overlay")
p = interactive(thresholding,_img = imgPathSelector)

rows = []
rows.append(HBox([p.children[0]]))
controls = VBox(rows, layout = Layout(display='flex'))
output = p.children[-1]
display(VBox([controls, output]))

VkJveChjaGlsZHJlbj0oVkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oU2VsZWN0KGRlc2NyaXB0aW9uPXUnRmlsZTonLCBsYXlvdXQ9TGF5b3V0KHdpZHRoPXUnMTAwJScpLCBvcHRpb27igKY=


In [790]:
vmapFiltered,_,_ = filterVmap(raw_vmap,[0.25,0.15,0.45,0.35])
vmapLessFiltered,_,_ = filterVmap(raw_vmap,[0.25,0.25,0.05,0.05])
pplot(vmapFiltered,"vmapFiltered")
pplot(vmapLessFiltered,"loose vmapFiltered")

In [791]:
h,w = vmapFiltered.shape[:2]
display2 = np.copy(vmapFiltered)
display1 = cv2.cvtColor(display2, cv2.COLOR_GRAY2BGR)
display2 = cv2.applyColorMap(display2,cv2.COLORMAP_PARULA)
display2 = cv2.cvtColor(display2,cv2.COLOR_RGB2BGR)
display3 = np.copy(display2)

t0 = time.time()
# lines = cv2.HoughLines(vmapFiltered,1,np.pi/180, 100)
# lines = cv2.HoughLines(raw_vmap,1,np.pi/180, 100)
# ----------------------------
bestM = 0.0
bestIntercept = 0
thetas = []; rhos = []
ms = []; bs = []

# validGnd, avgTheta = isGndPresent(lines)
# avgA = np.rad2deg(avgTheta)
# # ----------------------------
# if validGnd:
#     pts1 = []; pts2 = []
#     for i in range(0,len(lines)):
#         for r,theta in lines[i]:
#             angle = theta-np.pi
#             deg = np.rad2deg(angle)
#             if(avgA-2.0 <= deg <= avgA+2.0):
#                 thetas.append(theta)
#                 rhos.append(r)
#                 a = np.cos(theta);            b = np.sin(theta)
#                 x0 = a*r;                     y0 = b*r;
#                 x1 = int(x0 + 1000*(-b));     y1 = int(y0 + 1000*(a)) 
#                 x2 = int(x0 - 1000*(-b));     y2 = int(y0 - 1000*(a))
#                 pt1 = (x1,y1);      pt2 = (x2,y2)
#                 tpt1 = (x1,y1-20);     tpt2 = (x2,y2-20)
#                 m = float(y2 - y1)/float(x2 - x1)
#                 b = int(y1 - m * x1)
#                 if(b > bestIntercept):
#                     bestIntercept = b
#                     bestM = m
#                 ms.append(m)
#                 bs.append(b)
#                 pts1.append([x1,y1])
#                 pts2.append([x2,y2])
                    
#                 pts = [(0,h),pt1,pt2,(w, h),(0, h)]
#                 pts = np.array(pts)
# #                 cv2.fillPoly(display1, [pts], (0,0,0))
#                 cv2.line(display2,pt1, pt2, (0,255,0),1)
# #                 cv2.line(display1,pt1, pt2, (0,0,0),4)
# #                 cv2.circle(display2,(0,b),2,(255,0,255),-1)
#     gnd_fnd = True
# else:
#     gnd_fnd = False
#     print("No Ground Found")

validGnd,bestM,bestIntercept, sets = estimateGndLineCoeffsHough2(vmapFiltered)
t1 = time.time()
dt = t1 - t0
print("[%.2f] Method 2 = %.2f Hz" % (dt,1/dt))

lineParams = None
if validGnd:
    tmpy = int(w * bestM + bestIntercept)
    cv2.line(display2,(0,bestIntercept), (w,tmpy), (255,0,0),1)
    lineParams = [bestM,bestIntercept]
vmapNoGnd = cv2.cvtColor(display1,cv2.COLOR_BGR2GRAY)
# pplot(vmapNoGnd,"No Ground Line")
pplot(display2,"Ground Line 2")


[0.01] Method 2 = 142.83 Hz


In [792]:
gndImg = np.copy(vmapFiltered)
dispG = np.copy(display3)
h,w = gndImg.shape[:2]
nonzero = gndImg.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
nonzerox = np.reshape(nonzerox,(nonzerox.shape[0],1))
line_X = np.arange(nonzerox.min(), nonzerox.max())[:, np.newaxis]
plt.figure("RANSAC")
plt.clf()
plt.subplots_adjust(wspace=0.0,hspace=0.0,left=0.0,right=1.0,top=1.0, bottom=0.0)
plt.axis([0.0, w, h, 0])
plt.scatter(nonzerox,nonzeroy, color='yellowgreen', marker='.',label='Inliers')
tmpX = 48
for i in range(0,len(ms)):
    yhats = line_X * ms[i] + bs[i]
    plt.plot(line_X, yhats, color='red', linewidth=1)
    tmpY = tmpX * ms[i] + bs[i]
    plt.plot(tmpX, tmpY, color='cyan', marker='o', markersize=10)
    
plt.show()    

In [793]:
def obstacle_search_disparity(_vmap, x_limits, pixel_thresholds=(1,30), window_size=None, lineCoeffs=None, verbose=False, timing=False):
    flag_done = False
    try_last_search = False
    tried_last_resort = False
    flag_found_start = False
    flag_hit_limits = False
    count = 0; nWindows = 0; dt = 0
    yLims = []; windows = []

    try: img = cv2.cvtColor(_vmap,cv2.COLOR_GRAY2BGR)
    except:
        img = np.copy(_vmap)
        print("[WARNING] obstacle_search ------------  Unnecessary Image Color Converting")
    # Limits
    h,w = img.shape[:2]
    xmin, xmax = x_limits
    pxlMin, pxlMax = pixel_thresholds
    # Starting Pixel Coordinate
    yk = prev_yk = 0
    xk = (xmax + xmin) / 2
    
    if(lineCoeffs is not None): yf = int(xk * lineCoeffs[0] + lineCoeffs[1])
    else: yf = h
        
    # Window Size
    if(window_size is None):
        dWy = 10
        dWx = abs(xk - xmin)
        if(dWx <= 2):
            if xk <= 5: dWx = 1
            else:  dWx = 2
    else: dWx, dWy = np.int32(window_size)/2

    if(yk <= 0): yk = 0 + dWy
    if(verbose): print("Object Search Window [%d x %d] -- Starting Location (%d, %d)" % (dWx,dWy,xk, yk) )

    if(timing): t0 = time.time()

    # Grab all nonzero pixels
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Begin Sliding Window Search Technique
    while(not flag_done):
        if(xk >= w): # If we are at image edges we must stop
            flag_hit_limits = True
            if(verbose): print("[INFO] Reached max image width.")
        if((yk >= yf) and (lineCoeffs is not None)):
            flag_hit_limits = True
            if(verbose): print("[INFO] Reached Estimated Ground line.")
        elif(yk + dWy >= h):
            flag_hit_limits = True
            if(verbose): print("[INFO] Reached max image height.")
        
        if(flag_hit_limits): flag_done = True

        # Slide window from previousy found center (Clip at image edges)
        # Update vertical [Y] window edges
        if(yk - dWy >= 0): wy_low = yk - dWy
        else: wy_low = 0

        if(yk + dWy <= h): wy_high = yk + dWy
        else: wy_high = h

        # Update horizontal [X] window edges
        if(xk - dWx >= 0): wx_low = xk - dWx
        else: wx_low = 0

        if(xk + dWx <= w): wx_high = xk + dWx
        else: wx_high = w

        # Identify the nonzero pixels in x and y within the window
        good_inds = ((nonzeroy >= wy_low) & (nonzeroy < wy_high) &
                    (nonzerox >= wx_low) &  (nonzerox < wx_high)).nonzero()[0]
        nPxls = len(good_inds)
        if(verbose):
            print("Current Window [" + str(count) + "] ----- Center: " + str(xk) + ", " + str(yk) + " ----- # of good pixels = "  + str(nPxls))

        # Record mean coordinates of pixels in window and update new window center
        if(nPxls >= pxlMax):
            if(nWindows == 0):
                yLims.append(yk - dWy)
                if flag_hit_limits:
                    flag_done = False
                    print("\tTrying Last Ditch Search...")
            else:
                yLims.append(yk)
                try_last_search = False
            windows.append([(wx_low,wy_high),(wx_high,wy_low)])
            nWindows += 1
            prev_yk = yk
            flag_found_start = True
        elif(nPxls <= pxlMin and flag_found_start):
            flag_done = True
            yLims.append(prev_yk + dWy)
            
        # Update New window center coordinates
        xk = xk
        yk = yk + 2*dWy
        count += 1

    if(verbose): print("[%d] Good Windows" % nWindows)
    if(timing):
        t1 = time.time()
        dt = t1 - t0
        print("\t[obstacle_search] --- Took %f seconds to complete" % (dt))

    return yLims, windows, dt

def find_obstacles_disparity(vmap, dLims, xLims, search_thresholds = (3,30), ground_detected=True, lineCoeffs=None, verbose=False,timing=False):
    obs = []; obsUmap = []; windows = []; ybounds = []; dBounds = []
    nObs = len(dLims)

    if(timing): t0 = time.time()

    for i in range(nObs):
        xs = xLims[i]
        ds = dLims[i]
        ys,ws,_ = obstacle_search_disparity(vmap, ds, search_thresholds, lineCoeffs=lineCoeffs, verbose=verbose)
        # if self.debug_obstacle_search: print("Y Limits[",len(ys),"]: ", ys)
        if(len(ys) <= 2 and ground_detected):
            if(verbose): print("[INFO] Found obstacle with zero height. Skipping...")
        elif(len(ys) <= 1):
            if(verbose): print("[INFO] Found obstacle with zero height. Skipping...")
        elif(ys[0] == ys[1]):
            if(verbose): print("[INFO] Found obstacle with zero height. Skipping...")
        else:
            ybounds.append(ys)
            obs.append([
                (xs[0],ys[0]),
                (xs[1],ys[-1])
            ])
            obsUmap.append([
                (xs[0],ds[0]),
                (xs[1],ds[1])
            ])
            windows.append(ws)
            dBounds.append(ds)

    if(timing):
        t1 = time.time()
        dt = t1 - t0
        print("\t[INFO] find_obstacles() --- Took %f seconds (%.2f Hz) to complete" % (dt, 1/dt))
    return obs, obsUmap, ybounds, dBounds, windows, len(obs)

In [794]:
pplot(raw_umap,"umap")
# _,filtU = cv2.threshold(raw_umap, 24,255,cv2.THRESH_TOZERO)

filtU, strips, stripsT = filterUmap(raw_umap,[0.15,0.15,0.25,0.35])


kernelI = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
filtU = cv2.morphologyEx(filtU, cv2.MORPH_CLOSE, kernelI)
_,filtU2 = cv2.threshold(filtU, 1,255,cv2.THRESH_BINARY)

pplot(filtU2,"filtered2 umap")
pplot(filtU,"filtered umap")

filtered_contours,contours = vboat.find_contours(filtU,threshold = 100.0, max_thresh=-1)
print(len(filtered_contours))

7


In [796]:
copy = np.copy(curImg)
dispO = cv2.applyColorMap(copy,cv2.COLORMAP_PARULA)
copy = cv2.cvtColor(copy,cv2.COLOR_GRAY2BGR)
dispO = cv2.cvtColor(dispO,cv2.COLOR_RGB2BGR)

dispU = cv2.applyColorMap(filtU,cv2.COLORMAP_PARULA)
dispU = cv2.cvtColor(dispU,cv2.COLOR_RGB2BGR)
for cnt in filtered_contours:
    cv2.drawContours(dispU, [cnt], 0, (255,255,255), 1)
pplot(dispU,"Contours")

xLims, dLims, _ = vboat.extract_contour_bounds(filtered_contours)
obs, obsU, ybounds, dbounds, windows, nObs = find_obstacles_disparity(vmapLessFiltered, dLims, xLims, ground_detected=validGnd, lineCoeffs = lineParams, verbose=False)
# obs, obsU, ybounds, dbounds, windows, nObs = find_obstacles_disparity(raw_vmap, dLims, xLims, ground_detected=validGnd, lineCoeffs = lineParams, verbose=False)
# obs, obsU, ybounds, dbounds, windows, nObs = vboat.find_obstacles(vmapFiltered, dLims, xLims, ground_detected=gnd_fnd)

print(nObs)

# print(dLims)
# print(xLims)
for i in range(0,len(xLims)):
    avgX = int(np.mean(xLims[i]))
    avgD = int(np.mean(dLims[i]))
    cv2.circle(dispU,(avgX,avgD),2,(255,0,255),-1)
pplot(dispU,"Contours")

cpyV = np.copy(vmapFiltered)
dispWindows = cv2.applyColorMap(cpyV,cv2.COLORMAP_PARULA)
dispWindows = cv2.cvtColor(dispWindows,cv2.COLOR_RGB2BGR)

for wins in windows:
    for win in wins:
        cv2.rectangle(dispWindows,win[0],win[1],(255,255,0), 1)
if validGnd:
    tmpy = int(w * bestM + bestIntercept)
    cv2.line(dispWindows,(0,bestIntercept), (w,tmpy), (255,0,0),1)
pplot(dispWindows,"Windows")

alpha=0.35
[cv2.rectangle(copy, ob[0], ob[1],(255,0,0),3) for ob in obs]
cv2.addWeighted(copy, alpha, dispO, 1 - alpha, 0, dispO)
pplot(dispO,"Obstacles")


Object Search Window [4 x 10] -- Starting Location (251, 10)
Current Window [0] ----- Center: 251, 10 ----- # of good pixels = 0
Current Window [1] ----- Center: 251, 30 ----- # of good pixels = 0
Current Window [2] ----- Center: 251, 50 ----- # of good pixels = 0
Current Window [3] ----- Center: 251, 70 ----- # of good pixels = 0
Current Window [4] ----- Center: 251, 90 ----- # of good pixels = 0
Current Window [5] ----- Center: 251, 110 ----- # of good pixels = 0
Current Window [6] ----- Center: 251, 130 ----- # of good pixels = 0
Current Window [7] ----- Center: 251, 150 ----- # of good pixels = 0
Current Window [8] ----- Center: 251, 170 ----- # of good pixels = 0
Current Window [9] ----- Center: 251, 190 ----- # of good pixels = 0
Current Window [10] ----- Center: 251, 210 ----- # of good pixels = 0
Current Window [11] ----- Center: 251, 230 ----- # of good pixels = 0
Current Window [12] ----- Center: 251, 250 ----- # of good pixels = 0
Current Window [13] ----- Center: 251, 270 -

In [303]:
cnts = []
h,w = vmapFiltered.shape[:2]

display = np.copy(vmapFiltered)
display = cv2.cvtColor(display, cv2.COLOR_GRAY2BGR)

for i in range(0, len(pts1)):
    pts = []
    pts.append((0,h))
    pts.append(tuple(pts1[i]))
    pts.append(tuple(pts2[i]))
    pts.append((w, h))
    pts.append((0, h))
    pts = np.array(pts)
    cv2.fillPoly(display, [pts], (0,0,0))
vmapNoGnd = cv2.cvtColor(display,cv2.COLOR_BGR2GRAY)
pplot(vmapNoGnd,"No Ground Line")

In [279]:
cv2.fillPoly(display2, [pts], (0,0,0))
pplot(display2,"Ground Line 2")

In [293]:
display2 = np.copy(vmapFiltered)
# display2 = cv2.cvtColor(display2, cv2.COLOR_GRAY2BGR)
display2 = cv2.applyColorMap(display2,cv2.COLORMAP_PARULA)
display2 = cv2.cvtColor(display2,cv2.COLOR_RGB2BGR)
pplot(display2,"Ground Line 2")

In [269]:
pts1 = []
pts2 = []
for i in range(0,len(rhos)):
    tmpA = thetas[i]
    tmpR = rhos[i]
    m = (-np.cos(tmpA))/np.cos(tmpA)
    b = tmpR/np.sin(tmpA)
    x0 = 0
    y0 = int(m*x0 + b)

    y1 = 480
    x1 = int((y1 - b)/m)
    pts1.append([x0,y0])
    pts2.append([x1,y1])
    print((x0,y0), (x1,y1))
    cv2.line(display2,(x0,y0), (-x1,y1), (255,0,0),1)
pplot(display2,"Ground Line 2")

((0, 238), (-241, 480))
((0, 242), (-237, 480))
((0, 245), (-234, 480))
((0, 231), (-248, 480))
((0, 249), (-230, 480))
((0, 236), (-243, 480))
((0, 252), (-227, 480))


In [128]:
display = np.copy(vmapFiltered)
display = cv2.cvtColor(display, cv2.COLOR_GRAY2BGR)

t0 = time.time()
accumulator, thetas, rhos = hough_line(vmapFiltered)
show_hough_line(vmapFiltered, accumulator,thetas, rhos)

unis = np.unique(accumulator)
topUs = unis[-50:]
for u in topUs:
    idxs = np.where(accumulator == u)
    if(idxs[0].shape[0] == 1):
        tmpR = rhos[idxs[0]]
        tmpAng = thetas[idxs[1]]
        tmpDeg = np.rad2deg(tmpAng)
#         print("R = %.1f | Theta = %.2f" % (tmpR, tmpDeg))
        if(tmpDeg <= -20.0):
#             print("R = %.1f | Theta = %.2f" % (tmpR, tmpDeg))
            pt1, pt2 = extract_houghline(tmpR,tmpAng)
            cv2.line(display,pt1, pt2, (0,255,0),1)
    # print u, idxs[0], idxs[1]

t1 = time.time()
dt = t1 - t0
print("[%.2f] Method 1 = %.2f Hz" % (dt,1/dt))
    
pplot(display,"Ground Line")

544
[2.46] Method 1 = 0.41 Hz


In [ ]:
m,b, xs,ys = estimateGndLineCoeffs(vmapFiltered)
validGnd, pts = extractGndLine(m,b)
if(validGnd): vmapNoGnd, vizGndLine = removeGround(raw_vmap,pts)

In [ ]:
threshsU = [0.25, 0.3, 0.3, 0.8,0.8,0.8]
nThreshsU = int(math.ceil((self.dmax/256.0) * len(threshsU)))
stripsPu = []
stripsU = strip_image(raw_umap, nstrips=nThreshsU)
for i, strip in enumerate(stripsU):
    if i == 0:
        maxdisparity = np.max(raw_umap)
        stripThreshs = [0.15, 0.25, 0.075, 0.065,0.05,0.025]
        nSubStrips = int(math.ceil((self.dmax/256.0) * len(stripThreshs)))
        tmpStrip,_ = vboat.test_filter_first_umap_strip2(strip,maxdisparity,nSubStrips,timing=debug_timing)
        # print("tmpStrip Shape: %s" % str(tmpStrip.shape))
    elif i == 1:
        tmpStrip = self.second_umap_strip_filter(strip,timing=debug_timing)
        # print("tmpStrip Shape: %s" % str(tmpStrip.shape))
    else:
        tmpMax = np.max(strip)
        tmpThresh = threshsU[i] * tmpMax
        _, tmpStrip = cv2.threshold(strip, tmpThresh, 255,cv2.THRESH_TOZERO)
        # print("tmpStrip Shape: %s" % str(tmpStrip.shape))
    stripsPu.append(tmpStrip)

ksz1 = self.kszs[0]
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,ksz1)
if len(stripsPu)>=2:stripsPu[1] = cv2.morphologyEx(stripsPu[1], cv2.MORPH_CLOSE, kernel)
if len(stripsPu)>=3: stripsPu[2] = cv2.morphologyEx(stripsPu[2], cv2.MORPH_OPEN, kernel)

umap = np.concatenate(stripsPu, axis=0)

contours,raw_cnts = self.find_contours(umap, contour_thresh,debug=self.debug_contours)

In [ ]:
# ------------- GET NEW DEPTH IMAGE ------------------
rgb,depth = cam.read()
pplot(rgb,"rgb")
tmp = depth*0.001
loc = np.where(tmp == 0.0)
tmp[loc] = 1.0
dipsarity = (focal*baseline)/tmp
dipsarity[loc] = 0.0
disparity2uintGain = 255/np.max(dipsarity)
dipsarity = np.uint8(dipsarity*disparity2uintGain)
disparity = dipsarity

# kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))
# disparity = cv2.morphologyEx(disparity, cv2.MORPH_CLOSE, kernel)

pplot(disparity,"disparity")
raw_umap, raw_vmap, dt = vboat.get_uv_map(disparity)
umap = np.copy(raw_umap)
vmap = np.copy(raw_vmap)

pplot(vmap,"Raw Vmap")
cv2.rectangle(vmap,(0,0),(1, vmap.shape[0]),(0,0,0), cv2.FILLED)

# ------------- THRESHOLD V STRIPS ------------------
vMax = np.max(vmap)
nThreshs = int(math.ceil((vMax/256.0)) * 4)
print(vMax,nThreshs)

stripsV = strip_image(vmap, nstrips=nThreshs, horizontal_strips=False)
pplots(stripsV,"vmap strips",(1,nThreshs))

# thresholds = [0.95,0.35,0.35,0.5]
thresholds = [0.85,0.85,0.75,0.5]

stripsPV = []
for i, strip in enumerate(stripsV):
    tmpMax = np.max(strip)
    tmpMean = np.mean(strip)
    tmpStd = np.std(strip)
    if(tmpMean == 0):
        stripsPV.append(strip)
        continue
    tmp = (vMax - tmpMean)/tmpStd
    tmp2 = (vMax- tmpStd)/ tmpMean
    print("---------- [Strip %d] ---------- \r\n\tMax = %.1f, Mean = %.1f, Std = %.1f" % (i,tmpMax,tmpMean,tmpStd))
    dratio = vMax/255.0
    relRatio = (tmpMax-tmpStd)/float(vMax)
    rrelRatio = (tmpMean)/float(tmpMax)
    print("\tRatios: %.3f, %.3f, %.3f" % (dratio, relRatio, rrelRatio))
#     if(dratio >= 0.4): gain = (rrelRatio*thresholds[i] + relRatio*dratio)
#     else: gain = 1.0 - (rrelRatio + relRatio*dratio)*thresholds[i]
    # gain = (1.0 - (thresholds[i]*relRatio))*dratio
    if(relRatio >= 0.4): gain = relRatio + rrelRatio
    else: gain = 1.0 - (relRatio + rrelRatio)
    thresh = int(thresholds[i]* gain * tmpMax)
    print("\tGain = %.2f, Thresh = %d" % (gain,thresh))
    _, tmpStrip = cv2.threshold(strip, thresh,255,cv2.THRESH_TOZERO)
#     _, tmpStrip = cv2.threshold(strip, thresh,255,cv2.THRESH_BINARY)
    stripsPV.append(tmpStrip)

pplots(stripsPV,"thresholded vmap strips",(1,nThreshs))

filtV = np.concatenate(stripsPV, axis=1)
pplot(filtV,"Filtered Vmap")
# ------------- GROUND SEGMENTATION ------------------
groundSeg(filtV)

In [ ]:
groundSeg(filtV, stopP= 0.75,maxTrials=100)

In [95]:
def extract_houghline(rho, theta, r_offset = 0, verbose=False):
    angle = np.rad2deg(theta)
    if(verbose): print("Theta = %.3f | Radius = %.3f" % (angle, rho))
    r = rho + r_offset
    a = np.cos(theta);            b = np.sin(theta) 
    x0 = a*r;                     y0 = b*r;
    x1 = int(x0 + 1000*(-b));     y1 = int(y0 + 1000*(a)) 
    x2 = int(x0 - 1000*(-b));     y2 = int(y0 - 1000*(a))
    return (x1,y1), (x2,y2)

def get_ground_mask(_img, show_display = True, use_probabilistic = False, verbose = False):
    disp = np.copy(_img)
    disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
    display = np.copy(disp)
    
    cpy = np.copy(_img)
    thresh = int(np.max(cpy)*0.075)
    print(thresh)
    _,tmp = cv2.threshold(cpy, 32,255,cv2.THRESH_TOZERO)
    
    dt = 0
    t0 = time.time()
#     edges = cv2.Canny(tmp,50,150,apertureSize = 3)
    edges = np.copy(tmp)
    
    lines = cv2.HoughLines(edges,15,np.pi/180, 550)
    t1 = time.time()
    dt = t1 - t0
    # print("[INFO] HoughLines() --- Took %f seconds (%.2f Hz) to complete" % (dt, 1/dt))

    # t0 = time.time()
    # edges = cv2.Canny(tmp,1,150,apertureSize = 3)
    # linesP = cv2.HoughLinesP(edges,10,np.pi/180, 100,100,10)
    # t1 = time.time()
    # dt = t1 - t0
    # print("[INFO] HoughLinesProbabilistic() --- Took %f seconds (%.2f Hz) to complete" % (dt, 1/dt))

    pplot(edges,"edges")
#     print(len(lines))
    
    # for x1,y1,x2,y2 in linesP[0]:
    #     cv2.line(disp,(x1,y1),(x2,y2),(0,255,0),2)
    # for x1,y1,x2,y2 in linesP[1]:
    #     cv2.line(disp,(x1,y1),(x2,y2),(0,255,0),2)
    # for x1,y1,x2,y2 in linesP[2]:
    #     cv2.line(disp,(x1,y1),(x2,y2),(0,255,0),2)
    ############################################
    if(lines is None):
        print("No Lines found.")
        return False, _img, [], [], []
    numLines = len(lines)
    nLines = min(12, numLines)

    pts1 = []
    pts2 = []
    angles = []
    rhos = []
    for i in range(0,nLines):
        for r,theta in lines[i]:
            angle = np.rad2deg(theta)
            print("[%d] Theta = %.3f | Radius = %.3f" % (i, angle, r))
            if(show_display): 
                a = np.cos(theta);            b = np.sin(theta) 
                x0 = a*r;                     y0 = b*r;
                x1 = int(x0 + 1000*(-b));     y1 = int(y0 + 1000*(a)) 
                x2 = int(x0 - 1000*(-b));     y2 = int(y0 - 1000*(a))
                pt1 = (x1,y1);      pt2 = (x2,y2)
                cv2.line(display,pt1, pt2, (0,255,0),1)
            if(178.0 < angle <= 190.0): continue
            elif(-10.0 <= angle <= 10.0): continue
            elif(260.0 <= angle <= 280.0): continue
            elif(-100.0 <= angle <= -80.0): continue
            elif(-190.0 <= angle <= -170.0): continue
            else:
                if(verbose): print("[%d] Theta = %.3f | Radius = %.3f" % (i, angle, r))
                angles.append(angle)
                rhos.append(r)
                a = np.cos(theta);            b = np.sin(theta) 
                x0 = a*r;                     y0 = b*r;
                x1 = int(x0 + 1000*(-b));     y1 = int(y0 + 1000*(a)) 
                x2 = int(x0 - 1000*(-b));     y2 = int(y0 - 1000*(a))
                pt1 = (x1,y1);      pt2 = (x2,y2)
                pts1.append([x1,y1]);  pts2.append([x2,y2])
                if(verbose): print("[%d] Point1 = (%d, %d) | Point2 = (%d, %d)" % (i, x1,y1, x2,y2))
                if(show_display): cv2.line(display,pt1, pt2, (0,255,0),1)
    if(len(pts1) <= 0):
        print("[INFO] No Ground Present!")
        return False, cv2.cvtColor(disp,cv2.COLOR_BGR2GRAY), display, angles,rhos
    pts1 = np.array(pts1)
    pts2 = np.array(pts2)
    print(angles)
    idxMin = np.argmin(angles)
    idxMax = np.argmax(pts2[:,1])

    rgain = (rhos[idxMin]/10.0)
    again = (angles[idxMin]/180.0)
    print(rgain, again)
    tpt1, tpt2 = extract_houghline(rhos[idxMin],np.deg2rad(angles[idxMin]-again*0.75),-0.25*rgain)
#     tpt1, tpt2 = extract_houghline(rhos[idxMin],np.deg2rad(np.mean(angles)),-2)
    
    pts = np.array([
        (0, disp.shape[0]),
        (tpt1[0],tpt1[1]),
        (tpt2[0],tpt2[1]),
        (disp.shape[1], disp.shape[0]),
        (0, disp.shape[0])
    ])
    cv2.fillPoly(disp, [pts], (0,0,0))

    newVmap = cv2.cvtColor(disp,cv2.COLOR_BGR2GRAY)
    return True, newVmap, display, angles,rhos

def find_rough_ground_breakpoint(_img, verbose = False, debug = False,show_plots=False):
    h, _ = _img.shape[:2]
    hist = vertical_hist(_img)
    smHist = histogram_sliding_filter(hist,window_size=8)
    if(show_plots):
        plt.figure(1)
        plt.clf()
        plt.title('Histogram of the image')
        plt.plot(range(smHist.shape[0]), smHist[:])
    cnt = 0
    idxs = []
    lastIdx = None
    flag_started = False
    argIdx = smHist.shape[0]/4
    if(verbose): print("Starting Search Index: %d" % (argIdx))
    for i, val in enumerate(smHist[argIdx:]):
        idx = i+argIdx
        dval = val - smHist[idx-1]
        if(dval <= -15.0):
            if(lastIdx is None):
                lastIdx = idx
                idxs.append(idx)
            cnt += 1            
            flag_started = True
            if(verbose): print("[#%d] Found Negative Slope = %.1f" % (idx, dval))
        if(dval >= 0.0):
            if(flag_started):
                cnt = 0
                lastIdx = None
                if(debug): print("reseting")
                flag_started = False
        if(debug): print(idx,val, dval)
    if((cnt >= 1) and (lastIdx is not None)):
        if(debug): print("Found %d Indices with sufficient negative slopes: %s" % (len(idxs),str(idxs)))
        if(h-20 <= lastIdx <= h):
            try:
                lastIdx = idxs[-2]
                if(h-20 <= lastIdx <= h): lastIdx = -1
            except:
                lastIdx = -1
        if(verbose): print("cnt, lowest idx", cnt, lastIdx)
        return lastIdx
    else: return -1

def get_linear_ground_mask(img, debug_timings=True,show_plots=False,verbose=True):
        # img = np.copy(_img)
        h, _ = img.shape[:2]
        if(debug_timings):
            dt = 0
            t0 = time.time()
        
        lowIdx = find_rough_ground_breakpoint(img)
        if(lowIdx == -1): hm = h/2
        else: hm = lowIdx
        
        if(hm >= h-30): hm = h/2
    
        if(verbose): print("Thresholding Below %d" % (hm))
        topHalf = img[0:hm, :]
        botHalf = img[hm:h, :]
        _,ttHalf = cv2.threshold(topHalf, 225,255,cv2.THRESH_TOZERO)
        _,tbHalf = cv2.threshold(botHalf, 64,255,cv2.THRESH_TOZERO)
        tmp = np.concatenate((ttHalf,tbHalf), axis=0)

        nonzero = tmp.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        nonzerox = np.reshape(nonzerox,(nonzerox.shape[0],1))

        line_X = np.arange(nonzerox.min(), nonzerox.max())[:, np.newaxis]
        
#         lr = linear_model.LinearRegression()
#         lr.fit(nonzerox,nonzeroy)
#         line_y = lr.predict(line_X)

        ransac = linear_model.RANSACRegressor(stop_probability=0.85,max_trials=30)
        ransac.fit(nonzerox, nonzeroy)
        inlier_mask = ransac.inlier_mask_
        outlier_mask = np.logical_not(inlier_mask)

        line_y_ransac = ransac.predict(line_X)
        
        if(debug_timings):
            t1 = time.time()
            dt = t1 - t0
            print("[INFO] get_linear_ground_mask() --- Took %f seconds (%.2f Hz) to complete" % (dt, 1/dt))
            
        if(show_plots):
            pplot(tmp,"Image")
            plt.clf()
            pplot(tmp,"Image")

            plt.scatter(nonzerox,nonzeroy, color='yellowgreen', marker='.',label='Inliers')
            # plt.plot(line_X, line_y, color='white', linewidth=2, label='Linear regressor')
            plt.plot(line_X, line_y_ransac, color='red', linewidth=2, label='RANSAC regressor')
            
        m = ransac.estimator_.coef_
        b = ransac.estimator_.intercept_
        print("Estimated coefficients (RANSAC): m = %.3f | b = %.3f" % (m,b))
        return m,b, line_X,line_y_ransac

In [ ]:
idx = np.argmin(angs)
dum = cv2.cvtColor(tmp,cv2.COLOR_GRAY2BGR)
rgain = (rs[idx]/10.0)
again = (angs[idx]/180.0)
print(again)
print(rgain)

pt1, pt2 = extract_houghline(rs[idx],np.deg2rad(angs[idx]))
tpt1, tpt2 = extract_houghline(rs[idx],np.deg2rad(angs[idx]-again*0.75),-0.25*rgain)
cv2.line(dum,pt1, pt2, (0,255,0),1)
cv2.line(dum,tpt1, tpt2, (0,0,255),1)
pplot(dum,"test lines")

In [ ]:
rgb,depth = cam.read()
test_depth = np.copy(depth)

t0 = time.time()
norm = cv2.normalize(depth, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
loc = np.where(norm == 0.0)
norm[loc] = 1.0
disparity = (focal*baseline)/norm
disparity[loc] = 0.0
disparity = np.uint8(disparity)
dmax0 = np.max(disparity); dmin0 = np.min(disparity)
t1 = time.time()
dt = t1 - t0
print("[%.2f] Normalized Disparity Limits: %.2f, %.2f" % (1/dt,dmax0,dmin0))

t0 = time.time()
test_depth = test_depth*0.001
loc2 = np.where(test_depth == 0.0)
test_depth[loc2] = 1.0
testD = (focal*baseline)/test_depth
testD[loc2] = 0.0
disparity2uintGain = 255/np.max(testD)
testD = np.uint8(testD*disparity2uintGain)
dmax1 = np.max(testD); dmin1 = np.min(testD)
t1 = time.time()
dt = t1 - t0
print("[%.2f] Test Disparity Limits: %.2f, %.2f" % (1/dt, dmax1,dmin1))

pplot(depth,"raw depth")
pplot(norm,"normalized depth")
pplot(disparity,"disparity from norm")

pplot(testD,"test disparity")

In [ ]:
over = make_uv_overlay(dipsarity,raw_umap,raw_vmap)
pplot(over,'Overlay')

In [ ]:
rgb,depth = cam.read()
pplot(cv2.cvtColor(rgb,cv2.COLOR_RGB2BGR),"rgb")

tmp = depth*0.001
loc = np.where(tmp == 0.0)
tmp[loc] = 1.0
dipsarity = (focal*baseline)/tmp
dipsarity[loc] = 0.0
disparity2uintGain = 255/np.max(dipsarity)
dipsarity = np.uint8(dipsarity*disparity2uintGain)

# kernelI = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
# disparity = cv2.morphologyEx(dipsarity, cv2.MORPH_CLOSE, kernelI)
disparity = dipsarity
pplot(disparity,"disparity")
raw_umap, raw_vmap, dt = vboat.get_uv_map(disparity)
pplot(raw_umap,"Raw Umap")
pplot(raw_vmap,"Raw Vmap")

umap = np.copy(raw_umap)
vmap = np.copy(raw_vmap)

###########################################
cv2.rectangle(vmap,(0,0),(1, vmap.shape[0]),(0,0,0), cv2.FILLED)
# thresh0 = int(np.max(vmap)*0.025)
# _,tmp = cv2.threshold(vmap, thresh0,255,cv2.THRESH_TOZERO)

pplot(vmap,"Vmap")
display = np.copy(vmap)
display = cv2.cvtColor(display,cv2.COLOR_GRAY2BGR)
###########################################
m,b, xs,ys = get_linear_ground_mask(vmap,show_plots=True)
if(m >= 0.0):
    h,w = vmap.shape[:2]
    yf = int(m*w + b)
    pt1 = (0,int(b))
    pt2 = (w,yf-15)
    pts = np.array([(0, h),pt1,pt2,(w, h),(0, h)])
    cv2.fillPoly(display, [pts], (0,0,0))
    # cv2.line(display,pt1, pt2, (0,255,255),1)
    pplot(display,"ground line")
else: print("No Ground Detected.")
#######################################
# isGroundPresent, new_vmap, line_display, angs, rs = get_ground_mask(tmp)
# pplot(new_vmap,"new vmap")

In [ ]:
style = {'description_width': 'initial'}
layStretch = Layout(flex='1 1 auto')
layCenter = Layout(align_self='center',display = 'flex',align_items='stretch',flex='1 1 auto')
layWidth = Layout(width="auto",align_self='center')

lblSz1 = widgets.Label(value="Hough Thresholds", layout=layCenter)
sz1Selector = widgets.IntSlider(value=10,min=0,max=100,step=1,description=' ', layout = layCenter)
sz2Selector = widgets.IntSlider(value=550,min=0,max=1000,step=1,description=' ', layout = layCenter)

lblSz2 = widgets.Label(value="Probablistic Hough Thresholds:", layout=layCenter)
sz3Selector = widgets.IntSlider(value=100,min=0,max=1000,step=1,description='minLineLength', layout = layStretch)
sz4Selector = widgets.IntSlider(value=10,min=0,max=1000,step=1,description='maxLineGap', layout = layStretch)

lblU = widgets.Label(value="UMap Strip Thresholds (0,1):", layout=layCenter)
u1threshSelector = widgets.IntSlider(value=180,min=1,max=360,step=1,description='Angle',layout= layStretch)
# u1threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ',layout= layStretch)
u2threshSelector = widgets.FloatSlider(value=0.1,min=0,max=1,step=0.01,description=' ',layout = layStretch)

lblV = widgets.Label(value="VMap Strip Thresholds (0,1):", layout=layCenter)
v1threshSelector = widgets.IntSlider(value=5,min=0,max=255,step=1,description=' ',layout = layStretch)
v2threshSelector = widgets.IntSlider(value=70,min=0,max=255,step=1,description=' ',layout = layStretch)

lblD = widgets.Label(value="Deadzone Thresholds (U1, U2, V):", layout=layCenter)
d1threshSelector = widgets.FloatSlider(value=0.25,min=0,max=1,step=0.01,description=' ', layout = layStretch)
d2threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)
d3threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)

lblK = widgets.Label(value="Kernel Sizes:", layout=layCenter)
k11Selector = widgets.IntSlider(value=1,min=0,max=100,step=1,description='k11', layout = layStretch)
k12Selector = widgets.IntSlider(value=2,min=0,max=100,step=1,description='k12', layout = layStretch)
k21Selector = widgets.IntSlider(value=50,min=0,max=100,step=1,description='k21', layout = layStretch)
k22Selector = widgets.IntSlider(value=5,min=0,max=100,step=1,description='k22', layout = layStretch)
k31Selector = widgets.IntSlider(value=45,min=0,max=100,step=1,description='k31', layout = layStretch)
k32Selector = widgets.IntSlider(value=1,min=0,max=100,step=1,description='k32', layout = layStretch)

win1Selector = widgets.IntSlider(value=10,min=0,max=100,step=1,description='win1', layout = layStretch)
win1ThreshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)


flag1 = widgets.Checkbox(value=False, description='Use Test Pipeline', layout = layStretch)
flag2 = widgets.Checkbox(value=True, description='Show Computational Timing', layout = layStretch)
flagt2 = widgets.Checkbox(value=False, description='Show Misc Timings', layout = layStretch)
flag3 = widgets.Checkbox(value=True, description='Use Filtered Overlay', layout = layStretch)
flag4 = widgets.Checkbox(value=False, description='Show Colorful Obstacle Overlay', layout = layStretch)
# =================================================================================================
# 
# =================================================================================================
def thresholding(dx=0,dy=3, sz1=5,sz2=10,tu0=1,tu1=0.1,tv0=5, tv1=70, tdu = 0.4, tdu2 = 0.1, tdv = 0.15,
    k11=15,k12=2,k21=50,k22=5,k31=75,k32=1, win1=5, win1thr = 0.1, searchWin=10, flag_test=True,
    flag_timing=False,flag_timing2=False,flag3=True,flag4=True, avg_flag=False):
    
    _img = np.copy(vmap)
    disp = np.copy(_img)
    disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
    display = np.copy(disp)
    
    cpy = np.copy(_img)
    thresh = int(np.max(cpy)*0.075)
    print(thresh)
    _,tmp = cv2.threshold(cpy, 64,255,cv2.THRESH_TOZERO)
    
    dt = 0
    t0 = time.time()
    
#     edges = cv2.Canny(tmp,50,150,apertureSize = 3)
    edges = cpy
    if(flag_test): lines = cv2.HoughLinesP(edges,dx,np.pi/tu0, dy,sz1,sz2) 
    else: lines = cv2.HoughLines(edges,dx,np.pi/tu0, dy)
    # edges = cv2.Canny(tmp,50,150,apertureSize = 3)
    
    
#     t1 = time.time()
#     dt = t1 - t0
#     print("[INFO] HoughLines() --- Took %f seconds (%.2f Hz) to complete" % (dt, 1/dt))
#     print("[INFO] HoughLinesProbabilistic() --- Took %f seconds (%.2f Hz) to complete" % (dt, 1/dt))

#     pplot(edges,"edges")
#     print(len(lines))
    
    # for x1,y1,x2,y2 in linesP[0]:
    #     cv2.line(disp,(x1,y1),(x2,y2),(0,255,0),2)
    # for x1,y1,x2,y2 in linesP[1]:
    #     cv2.line(disp,(x1,y1),(x2,y2),(0,255,0),2)
    # for x1,y1,x2,y2 in linesP[2]:
    #     cv2.line(disp,(x1,y1),(x2,y2),(0,255,0),2)
    ############################################
    if(lines is None): print("No Lines found.")
    numLines = len(lines)
    nLines = min(6, numLines)

    pts1 = []
    pts2 = []
    angles = []
    rhos = []
    if(flag_test):
        for i in range(0,nLines):
            for x1,y1,x2,y2 in lines[i]:
                cv2.line(display,(x1,y1),(x2,y2),(0,255,0),2)
    else:
        for i in range(0,nLines):
            for r,theta in lines[i]:
                angle = np.rad2deg(theta)
                print("[%d] Theta = %.3f | Radius = %.3f" % (i, angle, r))
    #             a = np.cos(theta);            b = np.sin(theta) 
    #             x0 = a*r;                     y0 = b*r;
    #             x1 = int(x0 + 1000*(-b));     y1 = int(y0 + 1000*(a)) 
    #             x2 = int(x0 - 1000*(-b));     y2 = int(y0 - 1000*(a))
    #             pt1 = (x1,y1);      pt2 = (x2,y2)
    #             cv2.line(display,pt1, pt2, (0,255,0),1)
                if(178.0 <= angle <= 190.0): continue
                elif(-10.0 <= angle <= 10.0): continue
                elif(260.0 <= angle <= 280.0): continue
                elif(-100.0 <= angle <= -80.0): continue
                elif(-190.0 <= angle <= -170.0): continue
                elif(0.0 <= angle <= 60.0): continue
                else:
                    print("[%d] Theta = %.3f | Radius = %.3f" % (i, angle, r))
                    angles.append(angle)
                    rhos.append(r)
                    a = np.cos(theta);            b = np.sin(theta) 
                    x0 = a*r;                     y0 = b*r;
                    x1 = int(x0 + 1000*(-b));     y1 = int(y0 + 1000*(a)) 
                    x2 = int(x0 - 1000*(-b));     y2 = int(y0 - 1000*(a))
                    pt1 = (x1,y1);      pt2 = (x2,y2)
                    pts1.append([x1,y1]);  pts2.append([x2,y2])
                    print("[%d] Point1 = (%d, %d) | Point2 = (%d, %d)" % (i, x1,y1, x2,y2))
                    cv2.line(display,pt1, pt2, (0,255,0),1)
        pts1 = np.array(pts1)
        pts2 = np.array(pts2)
        print(angles)
        idxMin = np.argmin(angles)
        idxMax = np.argmax(pts2[:,1])

        rgain = (rhos[idxMin]/10.0)
        again = (angles[idxMin]/180.0)
        print(rgain, again)
        tpt1, tpt2 = extract_houghline(rhos[idxMin],np.deg2rad(angles[idxMin]-again*0.75),-0.25*rgain)
        cv2.line(display,tpt1, tpt2, (0,0,255),1)
    pplot(display,"display")
    
# =================================================================================================
# 
# =================================================================================================
p = interactive(thresholding,
    dx = sz1Selector, dy=sz2Selector, sz1=sz3Selector,sz2=sz4Selector,
    tu0 = u1threshSelector,tu1 = u2threshSelector, tv0 = v1threshSelector, tv1 = v2threshSelector,
    tdu = d1threshSelector, tdu2=d2threshSelector, tdv = d3threshSelector, k11=k11Selector,k12=k12Selector,
    k21=k21Selector,k22=k22Selector,k31=k31Selector,k32=k32Selector,win1=win1Selector,win1thr=win1ThreshSelector,
    searchWin=(0,40,1), flag_test = flag1, flag_timing = flag2, flag_timing2 = flagt2, flag3 = flag3, flag4 = flag4)

rows = []
rows.append(HBox([p.children[0]]))
rows.append(HBox([lblSz1,p.children[1], p.children[2]], layout = layStretch))
rows.append(HBox([lblSz2,p.children[3], p.children[4]], layout = layStretch))
rows.append(HBox([lblU, p.children[5], p.children[6]], layout = layStretch))
rows.append(HBox([lblV, p.children[7], p.children[8]], layout = layStretch))
rows.append(HBox([lblD, p.children[9], p.children[10], p.children[11]], layout = layStretch))
rows.append(HBox([p.children[12], p.children[13]], layout = layStretch))
rows.append(HBox([p.children[14], p.children[15]], layout = layStretch))
rows.append(HBox([p.children[16], p.children[17]], layout = layStretch))
rows.append(HBox([p.children[18], p.children[19]], layout = layStretch))

rows.append(HBox([p.children[-7],p.children[-6],p.children[-5], p.children[-4]], layout = layStretch))
rows.append(HBox([p.children[-3], p.children[-2]], layout = layStretch))

controls = VBox(rows, layout = Layout(display='flex'))
output = p.children[-1]

tabs = widgets.Tab([controls, output])
tabs.set_title(0, 'Parameters')
tabs.set_title(1, 'StdOuts')
# display(tabs)

display(VBox([controls, output]))